# Data Cleaning

## Introduction

* This notebook goes through a necasssary step of cleaning the data before it is used for exploratory data analysis. 
* The input of this notebook is a training dataset in csv format sourced from Kaggle. https://www.kaggle.com/c/nlp-getting-started
* The output of this notebook is a csv file with clean and lemmatized text data. 

## Reading and Understanding the dataset

In [27]:
# Importing libraries
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re           # regular expression 
import string       # String Handling
import random       #For selecting random rows
import nltk         # Natural langauage processing toolkit
from nltk.stem import WordNetLemmatizer  #Used for Lemmatizing the text
from nltk.corpus import wordnet          #Used for POS tagging 
from nltk.corpus import stopwords        #Stopwords to be removed from text


Reading the training data into a dataframe using pandas and viewing the top 20 rows of data

In [28]:
train_df = pd.read_csv("train.csv")
train_df.head(20)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


First 20 rows in 'Keyword' and 'Location' column are NaNs. Now Checking if there are nulls in other columns of the dataframe.

In [29]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Checking number of unique values in 'keywords' and 'location'.

In [30]:
len(train_df.keyword.unique())

222

In [31]:
len(train_df.location.unique())

3342

A glimpse at 30 rows from the 'locations' column selected randomly


In [32]:
random.sample(list((train_df.location.unique())),k=30)


['Host of #MindMoversPodcast',
 'United States of America',
 'New Jersey, usually',
 'Phoenix',
 'Victoria, Australia, Earth',
 '#expelcl*y',
 'Vancouver, Canada',
 ' New England',
 '3?3?7?SLOPelousas??2?2?5?',
 'CAMARILLO, CA',
 'Benton City, Washington',
 "wrapped arnd hyuk's finger",
 'Oklahoma',
 'Gurgaon, Haryana. ',
 'SEATTLE, WA USA',
 'lee london',
 'y/e/l',
 'Savannah, GA',
 'denver colorado',
 'Greenville',
 'amsterdayum 120615 062415',
 'Happily Married with 2 kids ',
 'Reading MA',
 'Australian Capital Territory',
 'Ellensburg to Spokane',
 ' ?currently writing a book?',
 'Chicago, IL',
 'Danville, VA',
 'Purgatory, USA',
 'Griffin :3']

A glimpse at 30 rows from the 'keywords' column selected randomly

In [33]:
random.sample(list((train_df.keyword.unique())),k=30)


['flames',
 'wreck',
 'blood',
 'casualty',
 'trapped',
 'devastation',
 'electrocute',
 'detonation',
 'outbreak',
 'emergency%20plan',
 'demolished',
 'detonate',
 'injury',
 'annihilated',
 'sinking',
 'disaster',
 'injuries',
 'drowned',
 'blew%20up',
 'sunk',
 'wreckage',
 'fire',
 'ruin',
 'rescued',
 'hazard',
 'wildfire',
 'rubble',
 'floods',
 'blown%20up',
 'aftershock']

A glimpse at 30 rows from the 'text' column selected randomly

In [34]:
# Selecting 30 random from text column randomly
random.sample(list((train_df.text)),k=30)

['@TemecaFreeman GM! I pray any attack of the enemy 2 derail ur destiny is blocked by the Lord &amp; that He floods ur life w/heavenly Blessings',
 'First Time Playing Hearthstone on PC Thoughts http://t.co/aBoLxMH1vy',
 "'We are now prepared to obliterate more rapidly and completely every productive enterprise the Japanese have above ground in any city.'",
 'i be on that hotboy shit',
 '@freefromwolves GodsLove &amp; #thankU brother Danny for RT of NEW VIDEO http://t.co/cybKsXHF7d The Coming Apocalyptic US Earthquake &amp; Tsunami',
 "#Greece's tax revenues collapse as debt crisis continues via @guardian #bailout http://t.co/cJvbQXw83s ^mp",
 'Black Eye 9: A space battle occurred at Star O784 involving 2 fleets totaling 3939 ships with 11 destroyed',
 "We're #hiring! Read about our latest #job opening here: RN Nurse Shift Manager Emergency Services - Full time... - http://t.co/sNuBZA6KSC",
 'My asshole is on fire  https://t.co/Y3FO0gHg8t',
 '#hot  C-130 specially modified to land in a

## Observation

Some data incosistencies or redundant information found in the dataset are as follows

* Upper case and lower case at unexpected location
* Punctuations
* Numbers in text 
* Use of cities, states and Country names. (Granularity problem)
* Special characters such as \x89ÛÒ and \n
* Hyperklinks
* Tags in tweets




## Cleaning the data

The method below does the following to clean anomalies in 'location' column:

* Changes all the text to lower case
* Removes punctutations
* Removes texts with numbers
* Removes cities names if country/state names are mentioned. (High level granularity is maintained)

In [35]:
# Method to chaange text to lower case and remove punctionation
def cleaning_location(text):
    text = str(text)
    text = text.lower()    #lower case
    text = text.split(',')[-1:][0].strip() # Removing city names when country/state name is present
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #removing punctuations
    text = re.sub('\w*\d\w*', '', text) #removing text with number
    return text

# Applying the method to location column
train_df.location = train_df.location.apply(cleaning_location)

The method below does the following to clean anomalies in 'keyword' column:

* Changes all the text to lower case
* Removes punctutations
* replaces number with space (as %20 was found in middle of two words)

In [36]:
def cleaning_keyword(text):
    text = str(text)
    text = text.lower()    ##lower case
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  ##removing punctuations
    text = re.sub('(\d+)', ' ', text)   #replacing numbers with space
    return text

# Applying the method to keyword column
train_df.keyword = train_df.keyword.apply(cleaning_keyword)

The following method to clean anomalies in 'text' column does the following

* Changes all the text to lower case
* Removes words starting with @ to remove the tags and mentions example: @barackobama
* Adds a column with hashtag values
* Removes links
* Removes punctuation
* Removes words with numbers
* Removes special characters examples: \x89û,\x89ûó etc 
* Removes '\n' 


In [37]:
def cleaning_text(text):
    
    text = str(text)
    text = text.lower()    ##lower case
    
    text = re.sub(r'@[A-Za-z]+[A-Za-z0-9-_]+', '',text) #removing any word starting with @   \w
    text = re.sub(r'https|www|http\S+', '', text)  #removing any word starting with http
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  ##removing punctuations
    text = re.sub('\w*\d\w*', '', text)  #removing words with numbers
    text = re.sub(r'[^\x00-\x7F]+', '', text) # removing special characters
    text.replace("\n","")
    return text

The following method converts NLTK tags to wordnet tags which would be used to lemmatize the words in the following method

In [38]:
def wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

The following method is used to lemmatize the sentences in the following order:

* Clean the sentences by calling the cleaning_text method on each sentence.
* Tokenizing the sentence and generating a nltk POS tag for each word in the cleaned sentence.
* Converting the nltk POS tag to Wordnet POS tag by calling wordnet_tag method.
* Removes the stopwords
* Lemmatizes the tokens using the pos tags and joining them to form a sentence of lemmatized words.


In [39]:
lemmatizing = WordNetLemmatizer()

def lemmatize_sentence(sentence):
    sentence = cleaning_text(sentence) #Cleaning the sentence
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  #Tokenizing and tagging each word 
    wordnet_tagged = map(lambda x: (x[0], wordnet_tag(x[1])), nltk_tagged)  # Coverting the NLTK tags to wordnet tag
    
    #Lemmatizing the tagged tokens
    lemmatized_sentence = [] #empty list for lemmatized words
    for word, tag in wordnet_tagged:
        if word not in set(stopwords.words('english')):  #removing stopwords
            if tag is None:                   
                lemmatized_sentence.append(word) #adding the word as it is if POS tag missing
            else:        
                #else use the tag to lemmatize the token
                lemmatized_sentence.append(lemmatizing.lemmatize(word, tag))  ##lemmatizing the token using the POS tag
    return " ".join(lemmatized_sentence)



In [40]:
# Applying the lemmatizing method to text column 
train_df.text= train_df.text.apply(lemmatize_sentence)

#Applying the lemmarizing methiod to keyword column
train_df.keyword = train_df.keyword.apply(lemmatize_sentence)

Let's have a look at the cleaned dataframe

In [41]:
train_df.sample(n=20)

,id,keyword,location,text,target
3258,4680,engulfed,nan,feel engulfed low selfimage take quiz,0
5327,7605,pandemonium,india,govt pass bill pandemonium upa use cant nda,0
5240,7493,obliteration,countries and counting,fear mind killer fear littledeath bring total ...,0
5706,8142,rescuer,nigeria,video pick body water rescuer search hundred m...,1
732,1058,bleeding,il,fell someone back hit head concrete bleed n shit,0
4448,6330,hostage,ca,wut lonely lunch get ditch im hungry hostage,0
5491,7835,quarantine,nan,reddit quarantine offensive content,0
1538,2221,chemical emergency,tx,emergency crew respond chemical spill downtown...,1
1861,2675,crush,nan,ron amp fez daves high school crush via,1
5849,8360,ruin,followurdreams my instagram,fresh da shower look still love new hair ruin ...,0


Checking if text from any row was completely removed due to cleaning

In [42]:
train_df.loc[train_df.text == '']

,id,keyword,location,text,target
5115,7295,nuclear reactor,nan,,0


There is one row with id 7295 where the whole text was removed. 

The original train data is read again to see the content of the row with id 7295

In [43]:
df_crosscheck = pd.read_csv('train.csv')
df_crosscheck.loc[df_crosscheck.id == 7295]

,id,keyword,location,text,target
5115,7295,nuclear%20reactor,NaN,Err:509,0


It can be observed above that the text in that particular row is an error message. Hence, that text was correctly cleaned from that row. 

The row is removed from the dataframe and the dataframe is saved as a csv file to be used for EDA.

In [44]:
train_df = train_df[train_df.text != '']
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7612 entries, 0 to 7612
Data columns (total 5 columns):
id          7612 non-null int64
keyword     7612 non-null object
location    7612 non-null object
text        7612 non-null object
target      7612 non-null int64
dtypes: int64(2), object(3)
memory usage: 356.8+ KB


In [45]:
train_df.to_csv("Clean_train_data.csv")

Cleaning test data using the predefined methods.

In [46]:
test_df = pd.read_csv('test.csv')

In [47]:
# Applying the method to location column
test_df.location = test_df.location.apply(cleaning_location)
# Applying the method to keyword column
test_df.keyword = test_df.keyword.apply(cleaning_keyword)

# Applying the lemmatizing method to text column 
test_df.text= test_df.text.apply(lemmatize_sentence)

# Applying the lemmatizing method to keyword column 
test_df.keyword= test_df.keyword.apply(lemmatize_sentence)



In [53]:
#Saving test dataset as csv
test_df.to_csv('Clean_test_data.csv')